In [1]:
from collections import namedtuple
import math
from math import sqrt
import numpy as np

import numpy as np
import z3
from z3 import *

def car(real, imag):
    return complex(real,imag)

def pol(rho, phi):
    return (rho, phi)

def car2pol(car):
    x, y = car.real, car.imag
    rho = np.sqrt(x**2 + y**2)
    phi = np.arctan2(y, x)
    phi = phi / 2 / math.pi * 360
    return(rho, phi)

def pol2car(rho, phi):
    phi = phi / 360 * 2 * math.pi
    x = rho * np.cos(phi)
    y = rho * np.sin(phi)
    return complex(x, y)

In [39]:
def checkcar(var):
    if isinstance(var, tuple):
        print(f'        converting {var} into carthesian to calculate with')
        return pol2car(*var)
    return var

def checkpol(var):
    if isinstance(var, complex):
        print(f'        converting {var} into polar to calculate with')
        return car2pol(var)
    if isinstance(var, int):
        print(f'        int found for {var} assuming angle = 0 for polar calculation')
        return (var, 0)
    return var

def parallel(x1,x2):
    return (x1*x2) / (x1 + x2)

def current(voltage, resistance):
    voltage = checkcar(voltage)
    cur = voltage / resistance
    print(f'    current in polar notation: {car2pol(cur)}')
    return cur

def voltage(current, resistance):
    current = checkcar(current)
    vol = current * resistance
    print(f'    voltage in polar notation: {car2pol(vol)}')
    return vol

def deg2rad(deg):
    return deg / 360 * 2 * math.pi

def rad2deg(rad):
    return rad / 2 / math.pi * 360


def Q(voltage, current):
    voltage = checkpol(voltage)
    current = checkpol(current)
    print(f'    Q = {voltage[0]:.2f} * {current[0]:.2f} * sin({voltage[1]:.2f} - {current[1]:.2f}) WARNING, if current enters at - should be -Q')
    return voltage[0] * current[0] * math.sin(deg2rad(voltage[1] - current[1]))
    
def P(voltage, current):
    voltage = checkpol(voltage)
    current = checkpol(current)
    # print(f'    U = {voltage}, I = {current}')
    print(f'    P = {voltage[0]:.2f} * {current[0]:.2f} * cos({voltage[1]:.2f} - {current[1]:.2f}) WARNING, if current enters at - should be -P')
    return voltage[0] * current[0] * math.cos(deg2rad(voltage[1] - current[1]))

def S(P, Q):
    return math.sqrt(P**2 + Q**2)



In [3]:
def capacitor_resistance(capacity, frequency=50):
    return -1 / (2*math.pi * frequency) / capacity

def capacitor_capacity(resistance, frequency=50):
    return 1 / (resistance * 2*math.pi * frequency)
    
def inductor_resistance(capacity, frequency=50):
    return (2*math.pi * frequency) * capacity

def inductor_capacity(resistance, frequency=50):
    return resistance / (2*math.pi*frequency)

In [4]:
def car2complexexpr(cart):
    return ComplexExpr(cart.real, cart.imag)

def pol2complexexpr(size, angle):
    return car2complexexpr(pol2car(size, angle))

def _to_complex(a):
    if isinstance(a, ComplexExpr):
        return a
    else:
        return ComplexExpr(a, RealVal(0))

def _is_zero(a):
    return (isinstance(a, int) and a == 0) or (is_rational_value(a) and a.numerator_as_long() == 0)

class ComplexExpr:
    def __init__(self, r, i):
        self.r = r
        self.i = i

    def __add__(self, other):
        other = _to_complex(other)
        return ComplexExpr(self.r + other.r, self.i + other.i)

    def __radd__(self, other):
        other = _to_complex(other)
        return ComplexExpr(other.r + self.r, other.i + self.i)

    def __sub__(self, other):
        other = _to_complex(other)
        return ComplexExpr(self.r - other.r, self.i - other.i)

    def __rsub__(self, other):
        other = _to_complex(other)
        return ComplexExpr(other.r - self.r, other.i - self.i)

    def __mul__(self, other):
        other = _to_complex(other)
        return ComplexExpr(self.r*other.r - self.i*other.i, self.r*other.i + self.i*other.r)

    def __mul__(self, other):
        other = _to_complex(other)
        return ComplexExpr(other.r*self.r - other.i*self.i, other.i*self.r + other.r*self.i)

    def inv(self):
        den = self.r*self.r + self.i*self.i
        return ComplexExpr(self.r/den, -self.i/den)

    def __div__(self, other):
        inv_other = _to_complex(other).inv()
        return self.__mul__(inv_other)

    def __rdiv__(self, other):
        other = _to_complex(other)
        return self.inv().__mul__(other)

    def __eq__(self, other):
        other = _to_complex(other)
        return And(self.r == other.r, self.i == other.i)

    def __neq__(self, other):
        return Not(self.__eq__(other))

    def simplify(self):
        return ComplexExpr(simplify(self.r), simplify(self.i))

    def repr_i(self):
        if is_rational_value(self.i):
            return "%s*I" % self.i
        else:
            return "(%s)*I" % str(self.i)

    def __repr__(self):
        if _is_zero(self.i):
            return str(self.r)
        elif _is_zero(self.r):
            return self.repr_i()
        else:
            return "%s + %s" % (self.r, self.repr_i())

def Complex(a):
    return ComplexExpr(Real('%s.r' % a), Real('%s.i' % a))
# I = ComplexExpr(RealVal(0), RealVal(1))

def evaluate_cexpr(m, e):
    return ComplexExpr(m[e.r], m[e.i])

def get_ans(m, var):
    r = evaluate_cexpr(m, var).r
    i = evaluate_cexpr(m, var).i
    real = float(r.numerator_as_long())/float(r.denominator_as_long())
    imag = float(i.numerator_as_long())/float(i.denominator_as_long())
    return complex(real, imag)

In [5]:
# template for AC nets
I1 = Complex("x")
I2 = Complex("y")
I3 = Complex("z")
s = Tactic('qfnra-nlsat').solver()





# s.add(ComplexExpr(cur.real, cur.imag) - I1 * ComplexExpr(8,4) - I2*ComplexExpr(0,-6) == 0)
# s.add(I2 * ComplexExpr(0,-6) - ComplexExpr(cur2.real, cur2.imag) == 0)
# s.add(I1 + I3 - I2 == 0)

s.add(100-I1*ComplexExpr(0,6.28)-I3*ComplexExpr(0,-6)==0)
s.add(I3*ComplexExpr(0,-6)+I2*10 == 0)
s.add(I1 + I2 -I3 == 0)
s.check()
m = s.model()

for var in [I1,I2, I3]:
    print(car2pol(get_ans(m,var)))
    

(30.864750979992415, -35.21359368159654)
(15.879783371533687, 85.75016285047698)
(26.466305619222812, -4.249837149523023)


In [6]:
# template for DC nets
I1 = Real('x')
I2 = Real('y')
I3 = Real('z')
I4 = Real('r')
I5 = Real('t')

s = Tactic('qfnra-nlsat').solver()
s.add(-10*I1-100-5*I2==0)
s.add(5*I2 + 100 -20*I4 == 0)
s.add(20*I4 + 40 == 0)

s.add(I1 - I2 - I3 == 0)
s.add(I3 - I4 - I5 == 0)
    
s.check()
m = s.model()

m.eval(I1), m.eval(I2), m.eval(I3),m.eval(I4),m.eval(I5)

(4, -28, 32, -2, 34)

In [7]:
Power = namedtuple('Power', 'Watt VAR')

def power(current, resistance):
    if isinstance(resistance, complex):
        return Power(power(current, resistance.real)[0], 
                     power(current, resistance.imag)[0])
    else:
        p = car2pol(current ** 2 * resistance)[0]
        print(f'    using I2* R: {current}**2 {resistance} = {p}')
        return Power(p,0)


def trans(primaryvoltage, primaryresistance, secondaryresistance, ratio):
    # bring everything to left side
    primaryvoltage = checkcar(primaryvoltage)
    res = primaryresistance + ratio ** 2 * secondaryresistance
    print(f'Total reactance for source {res}')
    Iprim = current(primaryvoltage, res)
    Isec = Iprim * ratio
    
    Uprim = checkcar(primaryvoltage) - primaryresistance * Iprim
    Usec  = Uprim / ratio
    print(f'Transformer: \nIp {car2pol(Iprim)}\nIs {car2pol(Isec)}\nUp {car2pol(Uprim)}\nUs {car2pol(Usec)}')
    print(f'Pprim {P(Uprim, Iprim)}W Qprim {Q(Uprim, Iprim)}VAR')
    psource = P(primaryvoltage, Iprim)
    qsource = Q(primaryvoltage, Iprim)
    print(f'Powers on primary side Psource: {psource}, Qsource {qsource}')
    print(f'Resistances on primary side: {power(Iprim, primaryresistance)}')
    print(f'Resistances on secondary side: {power(Isec, secondaryresistance)}')
    print(f'Efficiency transformer: {power(Isec, secondaryresistance).Watt} / {psource} = {power(Isec, secondaryresistance).Watt / psource}')

trans((100,0), 2, (100-200j), 0.1)

        converting (100, 0) into carthesian to calculate with
Total reactance for source (3-2.0000000000000004j)
    current in polar notation: (27.735009811261456, 33.6900675259798)
Transformer: 
Ip (27.735009811261456, 33.6900675259798)
Is (2.7735009811261455, 33.6900675259798)
Up (62.017367294604234, -29.744881296942225)
Us (620.1736729460423, -29.74488129694223)
        converting (53.846153846153854-30.769230769230774j) into polar to calculate with
        converting (23.076923076923073+15.384615384615387j) into polar to calculate with
    U = (62.017367294604234, -29.744881296942225), I = (27.735009811261456, 33.6900675259798)
    P = 62.02 * 27.74 * cos(-29.74 - 33.69) WARNING, if current enters at - should be -P
        converting (53.846153846153854-30.769230769230774j) into polar to calculate with
        converting (23.076923076923073+15.384615384615387j) into polar to calculate with
    Q = 62.02 * 27.74 * sin(-29.74 - 33.69) WARNING, if current enters at - should be -Q
Ppr

### Asynchronous machines

In [8]:
from math import sqrt
def phase2linevoltage(phasevoltage):
    return sqrt(3) * phasevoltage
def line2phasevoltage(linevoltage):
    return linevoltage / sqrt(3)
line2phasevoltage(10000)

def S3f(Ul, I):
    return sqrt(3) * Ul * I

def Pdriehoek(Pster):
    return 3 * Pster

In [9]:
def nullast(P,U,I):
    S = U * I # schijnbaar vermogen
    Q = (S**2 - P**2)**0.5 # blind vermogen
    # nullasten staan parallel aan de bron dus P = U**2 / R gebruiken
    Rij = U**2 / P  
    Xmu = U**2 / Q
    phi = rad2deg(math.acos((P / (U * I)))) # from formula P = U * I * cos(phi)
    print(
        'Nullast:\n'
        f'S = {S}\n'
        f'Q = {Q} : sqrt(S**2 - P**2) = sqrt({S**2} - {P**2})\n'
        f'phi = {phi} : rad2deg(acos((P / (U * I))))\n'
        f'Rij = {Rij} : U**2 / P\n'
        f'Xmu = {Xmu} : U**2 / Q\n')
    return Rij, Xmu


def kortsluit(P,U,I):
    S = U * I # schijnbaar vermogen
    Q = (S**2 - P**2)**0.5 # blind vermogen
    # nullasten staan in serie met bron dus P = I**2 * R gebruiken
    Rk = P / I**2
    Xk = Q / I**2
    phi = rad2deg(math.acos((P / (U * I)))) # from formula P = U * I * cos(phi)
    print(
        'Kortsluit:\n'
        f'S = {S}\n'
        f'Q = {Q} : sqrt(S**2 - P**2) = sqrt({S**2} - {P**2})\n'
        f'phi = {phi} : rad2deg(acos((P / (U * I))))\n'
        f"Rk = {Rk} : P / I ** 2. Note Rk = R1 + R'2\n"
        f'Xk = {Xk} : Q / I ** 2\n')
    return Rk, Xk



In [10]:
def slip(fnet, frotor):
    # slip = 1, means rotor is stationary
    # slip = 0, means fnet == frotor
    # slip < 0, means machine is acting as generator
    return (fnet - frotor) / fnet
slip(50,0), slip(50,50), slip(50, 60)

def slip2freq(slip, fnet=None):
    if not fnet: fnet = 50
    # slip = (fnet - frot) / fnet
    return -(slip * fnet - fnet)
slip2freq(0.0212)

48.94

In [11]:
def R2(Rk, ohmmeasured):
    # vervangingsweerstand as berekenen bij bekend zijn van Rk vanuit kortsluitproef
    return Rk - (ohmmeasured / 2)

def Ras(R2, slip):
    # vervangingsweerstand as berekenen inclusief slip
    return R2 * (1-slip) / slip

def Pas(I, R2, slip):
    # asvermogen bepalen
    return 3 * I**2 * Ras(R2, slip)

def koppel(P, freq):
    # P = w T
    return P / (freq * 2 * math.pi)

In [12]:
def asynchrone(voltage, Xk, Rk, R2, slip):
    # gegeven vervangingsschema en een slip bepaal stroom, vermogens en rendement
    # returns asvermogen
    if isinstance(voltage, int):
        voltage = (voltage, 0)
    reactance = complex(Rk + Ras(R2, slip), Xk)
    cur = checkpol(current(voltage, reactance))
    print('Asynchronous machine')
    print('Total reactance = ', reactance)
    print(f'Current = {cur} (voltage / reactance)')
    print('P = ', -3* P(voltage, cur))
    print('Q = ', -3*Q(voltage, cur))
    print('Pas = ', Pas(cur[0], R2, slip))
    print('rendement bij slip > 0 (anders 1/x doen)', Pas(cur[0], R2, slip) / (3 * P(voltage, cur)))
    return Pas(cur[0], R2, slip)
    
    
asynchrone((230,0), 0.2, 0.206, 0.156, -0.033333)

        converting (230, 0) into carthesian to calculate with
    current in polar notation: (49.62924367654542, -177.52658296177944)
        converting (-49.583006676742166-2.141792893831239j) into polar to calculate with
Asynchronous machine
Total reactance =  (-4.630046800468004+0.2j)
Current = (49.62924367654542, -177.52658296177944) (voltage / reactance)
    U = (230, 0), I = (49.62924367654542, -177.52658296177944)
    P = 230.00 * 49.63 * cos(0.00 - -177.53) WARNING, if current enters at - should be -P
P =  34212.274606952094
    Q = 230.00 * 49.63 * sin(0.00 - -177.53) WARNING, if current enters at - should be -Q
Q =  -1477.8370967435362
Pas =  -35734.44681659795
    U = (230, 0), I = (49.62924367654542, -177.52658296177944)
    P = 230.00 * 49.63 * cos(0.00 - -177.53) WARNING, if current enters at - should be -P
rendement bij slip > 0 (anders 1/x doen) 1.0444919908755947


-35734.44681659795

### Synchnonous machines

In [20]:


# def tas(slip, fnet):
#     was = 2 * math.pi * fnet * (1-slip)
#     print(slip)
#     return p(slip) / was
# s = np.linspace(-0.99,0.99,20)
# fnet = 50
# t = [tas(x, fnet) for x in s]

In [21]:
def X1(Unom, I2nullast, I1kortsluit, I2kortsluit):
    lamb = Unom / I2nullast
    print(f'Bij een I2 van {I2nullast}A wordt de nominale fasespanning gehaald van {Unom}V')
    I1nullast = I1kortsluit / I2kortsluit * I2nullast
    print(f'Bij een I2 van {I2kortsluit}A treedt een I1 op van {I1kortsluit}A, dus bij I2 van {I2nullast}A is I1 {I1nullast}A')
    X1 = Unom / I1nullast
    print('Hieruit volgt:')
    print(f'X1 = {X1} Ohm ({Unom} / {I1nullast})')
    print(f'Lambda = {lamb} ({Unom} / {I2nullast})')
    return X1

X1(230, 4,70, 12)

Bij een I2 van 4A wordt de nominale fasespanning gehaald van 230V
Bij een I2 van 12A treedt een I1 op van 70A, dus bij I2 van 4A is I1 23.333333333333332A
Hieruit volgt:
X1 = 9.857142857142858 Ohm (230 / 23.333333333333332)
Lambda = 57.5 (230 / 4)


9.857142857142858

In [ ]:
# capacitive behavior = phase angle current > phase angle voltage, like a balloon, where flow goes before size
# inductive behavior = phase angle voltage > current, like a train where voltage goes before motion

In [1]:
def behavior(phi):
    phi = (phi + 360) % 360
    angle_induccapa = 'capacitive' if phi <= 180 else 'inductive'
    angle_motorgenerator = 'generator' if 90 <= phi <= 270 else 'motor'
    return angle_induccapa, angle_motorgenerator

def cosphi_to_options(cosphi):
    # if you don't know if cosphi is for Inet or I1, just enter the cos phi and it will explain how it behaves
    phi = math.acos(cosphi) / 2 / math.pi * 360
    print(f'phi = {phi}, but could also have been {-phi} or with 180 degrees offset if we looked the other way')
    for p in (phi, -phi):
        for reverse in (0,180):
            newphi =  p + reverse
            print(f'{newphi} leads to {behavior(newphi)}')
cosphi_to_options(0.45)

NameError: name 'math' is not defined

In [91]:
def syn_net(Unet, X1, l = None):
    Iz = car2pol(current(U1, resistance_island)) # calculate Iz from U1 and resistance
    I1 = (Iz[0], (Iz[1] + 180) % 360) # I1 == Iz, but reverse phase angle
    
    UX1 = voltage(I1, complex(0,X1)) 
    print(f'Voltage over X1 {car2pol(UX1)}')
    U12 = car2pol(pol2car(*U1) - UX1) # Kirchhoff to determine U12
    print(f'U12 = Unet - UX1 --> {U1} - {car2pol(UX1)} = {U12}')
    print(f'Pas is {3 * P(U12, I1)}')
    QX1 = 3 * I1[0]**2 * X1
    Qas = 3 * Q(U12, I1)
    print(f'Qas = {Qas}')
    print(f'QX1 = {QX1} (3 * I**2 * X1)')
    print(f'Pnet = {-3 * P(U1, I1)}')
    print(f'Qnet = {-3 * Q(U1, I1)}')
    machineQ = QX1 + Qas
    induccapa = 'inductive' if machineQ > 0 else 'capacitive'
    print(f'machine is operating {machineQ}, therefore {induccapa}')
    delta = U12[1] - U1[1]
    motorgenerator = 'generator' if delta >= 0 else 'motor'
    print(f'lasthoek delta = {delta}, therefore {motorgenerator} behavior')
    phi = I1[1] % 360
    print(f'asserting behavior phase angle for I1 (phi): {phi}. 0-180=cap, 180-360=ind, 0-90 & 270-360 = motor, 90-270=generator')
    angle_induccapa = 'capacitive' if phi <= 180 else 'inductive'
    angle_motorgenerator = 'generator' if 90 <= phi <= 270 else 'motor'
    assert induccapa == angle_induccapa and motorgenerator == angle_motorgenerator
    
    if l:
        I2 = U12[0] / l
        print(f'I2 is {I2}A')
    return U12

phi = 63.256316049597, but could also have been -63.256316049597 or with 180 degrees offset if we looked the other way
63.256316049597 leads to ('capacitive', 'motor')
243.256316049597 leads to ('inductive', 'generator')
-63.256316049597 leads to ('inductive', 'motor')
116.74368395040301 leads to ('capacitive', 'generator')


In [93]:
acos(0.45) / 2 / math.pi * 360

63.256316049597

In [94]:
def syn_island(resistance_island, U1, X1, l = None):
    Iz = car2pol(current(U1, resistance_island)) # calculate Iz from U1 and resistance
    I1 = (Iz[0], (Iz[1] + 180) % 360) # I1 == Iz, but reverse phase angle
    
    UX1 = voltage(I1, complex(0,X1)) 
    print(f'Voltage over X1 {car2pol(UX1)}')
    U12 = car2pol(pol2car(*U1) - UX1) # Kirchhoff to determine U12
    print(f'U12 = Unet - UX1 --> {U1} - {car2pol(UX1)} = {U12}')
    print(f'Pas is {3 * P(U12, I1)}')
    QX1 = 3 * I1[0]**2 * X1
    Qas = 3 * Q(U12, I1)
    print(f'Qas = {Qas}')
    print(f'QX1 = {QX1} (3 * I**2 * X1)')
    print(f'Pnet = {-3 * P(U1, I1)}')
    print(f'Qnet = {-3 * Q(U1, I1)}')
    machineQ = QX1 + Qas
    induccapa = 'inductive' if machineQ > 0 else 'capacitive'
    print(f'machine is operating {machineQ}, therefore {induccapa}')
    delta = U12[1] - U1[1]
    motorgenerator = 'generator' if delta >= 0 else 'motor'
    print(f'lasthoek delta = {delta}, therefore {motorgenerator} behavior')
    phi = I1[1] % 360
    print(f'asserting behavior phase angle for I1 (phi): {phi}. 0-180=cap, 180-360=ind, 0-90 & 270-360 = motor, 90-270=generator')
    assert behavior(phi) == (induccapa, motorgenerator) 
    
    if l:
        I2 = U12[0] / l
        print(f'I2 is {I2}A')
    return U12


syn_island(3.38-2.552j, (230,0), 14, 100)

        converting (230, 0) into carthesian to calculate with
    current in polar notation: (54.30648237733567, 37.053884216260165)
        converting (54.30648237733567, 217.05388421626017) into carthesian to calculate with
    voltage in polar notation: (760.2907532826995, -52.946115783739835)
Voltage over X1 (760.2907532826995, -52.946115783739835)
U12 = Unet - UX1 --> (230, 0) - (760.2907532826995, -52.946115783739835) = (648.2317649175521, 110.6047499652292)
    P = 648.23 * 54.31 * cos(110.60 - 217.05) WARNING, if current enters at - should be -P
Pas is -29904.827445946677
    Q = 648.23 * 54.31 * sin(110.60 - 217.05) WARNING, if current enters at - should be -Q
Qas = -101287.11970449632
QX1 = 123866.14918439454 (3 * I**2 * X1)
    P = 230.00 * 54.31 * cos(0.00 - 217.05) WARNING, if current enters at - should be -P
Pnet = 29904.82744594667
    Q = 230.00 * 54.31 * sin(0.00 - 217.05) WARNING, if current enters at - should be -Q
Qnet = -22579.029479898214
machine is operating 2257

(648.2317649175521, 110.6047499652292)

In [96]:
from functools import partial
p = partial(asynchrone, 233, 78, 45, 10)

In [97]:
import altair as alt
import numpy as np
import pandas as pd


source = pd.DataFrame({
  's': s,
  't': t
})

alt.Chart(source).mark_line().encode(
    x='s',
    y='t'
)

NameError: name 't' is not defined